# Função para receber o gênero de uma música do dataset de acordo com seu ID

In [ ]:
import pandas as pd

def check_gender_by_id(filename, id, genders) -> None | str:
    df = pd.read_excel(filename)

    genero = df.loc[df['id'] == id, 'genero']

    if not genero.empty:
        if genero.iloc[0] in genders:
            return genero.iloc[0]
        else:
            return None
    else:
        return "ID not found"

In [ ]:
# Exemplo de uso
arquivo = '/content/generos.xlsx'
id_musica = 17  # Coloque o ID desejado
genders = ['Rock', 'Jazz', 'Pop', 'Classical', 'Country', 'Eletronical']
genero = check_gender_by_id(arquivo, id_musica, genders)
print(f'O gênero da música com ID {id_musica} é: {genero}')

O gênero da música com ID 17 é: Rock


In [ ]:
def dividir_musicas_em_listas(arquivo_xlsx, generos_validos):
    # Carrega o arquivo Excel para um DataFrame
    df = pd.read_excel(arquivo_xlsx)

    # Dicionário para armazenar as músicas de cada gênero
    generos = {}

    # Itera por todas as músicas no arquivo
    for id_musica in df['id']:
        # Usa a função check_gender_by_id para obter o gênero da música
        genero = check_gender_by_id(arquivo_xlsx, id_musica, generos_validos)

        # Se o gênero for válido (não None e dentro da lista de gêneros válidos), adiciona o ID
        if genero and genero in generos_validos:
            if genero not in generos:
                generos[genero] = []
            generos[genero].append(id_musica)

    # Listas para armazenar as músicas de treino, validação e teste
    musicas_treino = []
    musicas_validacao = []
    musicas_teste = []

    # Agora, vamos dividir as músicas por gênero em treino, validação e teste de forma aleatória
    for genero, musicas in generos.items():
        # Embaralha as músicas para aleatoriedade
        random.shuffle(musicas)

        # Verifica se há pelo menos 30 músicas para dividir
        if len(musicas) >= 30:
            # Divide as músicas de forma aleatória
            treino = musicas[:21]
            validacao = musicas[21:27]
            teste = musicas[27:30]

            # Adiciona as listas para cada categoria
            musicas_treino.extend(treino)
            musicas_validacao.extend(validacao)
            musicas_teste.extend(teste)
        else:
            # Se houver menos de 30 músicas, podemos decidir o que fazer, mas aqui estamos ignorando as músicas
            pass

    # Retorna as 3 listas
    return musicas_treino, musicas_validacao, musicas_teste

In [ ]:
import random
treino,validacao,teste = dividir_musicas_em_listas('/content/generos.xlsx',genders)

In [ ]:
for x in treino:
  print(check_gender_by_id('/content/generos.xlsx',x,genders))

Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country


In [ ]:
!unzip "/content/espectrogramas.zip"

Archive:  /content/espectrogramas.zip
replace espectrogramas/1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: espectrogramas/1.png    
  inflating: espectrogramas/10.png   
  inflating: espectrogramas/100.png  
  inflating: espectrogramas/101.png  
  inflating: espectrogramas/102.png  
  inflating: espectrogramas/103.png  
  inflating: espectrogramas/104.png  
  inflating: espectrogramas/105.png  
  inflating: espectrogramas/106.png  
  inflating: espectrogramas/107.png  
  inflating: espectrogramas/108.png  
  inflating: espectrogramas/109.png  
  inflating: espectrogramas/11.png   
  inflating: espectrogramas/110.png  
  inflating: espectrogramas/111.png  
  inflating: espectrogramas/112.png  
  inflating: espectrogramas/113.png  
  inflating: espectrogramas/114.png  
  inflating: espectrogramas/115.png  
  inflating: espectrogramas/116.png  
  inflating: espectrogramas/117.png  
  inflating: espectrogramas/118.png  
  inflating: espectrogramas/119.png  
  inflating: espec

In [ ]:
# Exemplo: Gêneros associados aos IDs
id_to_genre = {
    0: "Rock",
    1: "Pop",
    2: "Jazz",
    3: "Eletronical",
    4: "Classical",
    5: "Country"
    # Adicione todos os IDs e seus gêneros
}

# Converta os gêneros para índices numéricos para o treinamento
genre_to_index = {genre: idx for idx, genre in enumerate(set(id_to_genre.values()))}

# Exemplo: rock -> 0, pop -> 1, jazz -> 2
id_to_label = {id: genre_to_index[genre] for id, genre in id_to_genre.items()}


In [ ]:
import tensorflow as tf
import os


base_dir = "/content/espectrogramas"

def id_to_filepath(id):
  return os.path.join(base_dir,f"{id}.png")

In [ ]:
df = pd.read_excel('/content/generos.xlsx')

# Cria o mapeamento de gêneros para índices
genre_to_index = {genre: idx for idx, genre in enumerate(set(df['genero']))}

# Mapeia IDs das músicas aos índices das classes
id_to_label = {row['id']: genre_to_index[row['genero']] for _, row in df.iterrows()}

print("id_label:", id_to_label)

id_label: {1: 6, 2: 6, 3: 6, 4: 6, 5: 6, 6: 6, 7: 6, 8: 6, 9: 6, 10: 6, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 10, 22: 10, 23: 10, 24: 10, 25: 10, 26: 10, 27: 10, 28: 10, 29: 10, 30: 10, 31: 7, 32: 7, 33: 7, 34: 7, 35: 7, 36: 7, 37: 7, 38: 7, 39: 7, 40: 7, 41: 9, 42: 9, 43: 9, 44: 9, 45: 9, 46: 9, 47: 9, 48: 9, 49: 9, 50: 9, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 2, 62: 2, 63: 2, 64: 2, 65: 2, 66: 2, 67: 2, 68: 2, 69: 2, 70: 2, 71: 8, 72: 8, 73: 8, 74: 8, 75: 8, 76: 8, 77: 8, 78: 8, 79: 8, 80: 8, 81: 4, 82: 4, 83: 4, 84: 4, 85: 4, 86: 4, 87: 4, 88: 4, 89: 4, 90: 4, 91: 3, 92: 3, 93: 3, 94: 3, 95: 3, 96: 3, 97: 3, 98: 3, 99: 3, 100: 3, 101: 5, 102: 5, 103: 5, 104: 5, 105: 5, 106: 5, 107: 5, 108: 5, 109: 5, 110: 5, 111: 6, 112: 6, 113: 6, 114: 6, 115: 6, 116: 6, 117: 6, 118: 6, 119: 6, 120: 6, 121: 6, 122: 6, 123: 6, 124: 6, 125: 6, 126: 6, 127: 6, 128: 6, 129: 6, 130: 6, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0,

In [ ]:
def create_tf_dataset_with_labels(id_list, id_to_label):
    filepaths = [id_to_filepath(id) for id in id_list]
    labels = [id_to_label[id] for id in id_list]

    # Cria um dataset contendo pares (caminho, rótulo)
    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))

    # Carrega a imagem e retorna (imagem, rótulo)
    def load_image_and_label(filepath, label):
        image = tf.io.read_file(filepath)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [224, 224])  # Tamanho esperado pela CNN
        return image, label

    dataset = dataset.map(load_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset


In [ ]:
def normalizar_imagem(imagem):
    return tf.cast(imagem, tf.float32) / 255.0  # Normaliza para o intervalo [0, 1]

train_dataset = create_tf_dataset_with_labels(treino, id_to_label)
val_dataset = create_tf_dataset_with_labels(validacao, id_to_label)
test_dataset = create_tf_dataset_with_labels(teste, id_to_label)

train_dataset = train_dataset.map(lambda imagem, label: (normalizar_imagem(imagem), label))
val_dataset = val_dataset.map(lambda imagem, label: (normalizar_imagem(imagem), label))
test_dataset = test_dataset.map(lambda imagem, label: (normalizar_imagem(imagem), label))


# Agrupar em lotes e otimizar
batch_size = 32
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:
import cv2 as cv

img = cv.imread("/content/espectrogramas/125.png")


input_shape = img.shape
print(input_shape)
num_classes = len(set(id_to_label.values()))
print(f"Quantidade de classes: {num_classes}")

print(num_classes)

(924, 2325, 3)
Quantidade de classes: 11
11


In [ ]:
import os
from PIL import Image

def verificar_tamanhos_canais_pasta(pasta):
    tamanhos = {}
    canais = {}
    arquivos_verificados = 0

    for root, _, files in os.walk(pasta):
        for file in files:
            if file.endswith((".png", ".jpg", ".jpeg")):  # Ajuste as extensões conforme necessário
                caminho_completo = os.path.join(root, file)
                try:
                    with Image.open(caminho_completo) as img:
                        tamanho = img.size  # (largura, altura)
                        modo = img.mode  # Ex.: "L", "RGB", "RGBA"
                        canais_count = {'L': 1, 'RGB': 3, 'RGBA': 4}.get(modo, "Desconhecido")

                        tamanhos[caminho_completo] = tamanho
                        canais[caminho_completo] = canais_count
                        arquivos_verificados += 1
                except Exception as e:
                    print(f"Erro ao processar {caminho_completo}: {e}")

    if arquivos_verificados == 0:
        print("Nenhum arquivo de imagem foi encontrado na pasta especificada.")
        return

    # Identificar se todos têm o mesmo tamanho e canais
    tamanhos_unicos = set(tamanhos.values())
    canais_unicos = set(canais.values())

    print(f"\nTotal de imagens verificadas: {arquivos_verificados}")
    if len(tamanhos_unicos) == 1:
        print(f"Todos os espectrogramas têm o mesmo tamanho: {tamanhos_unicos.pop()}")
    else:
        print("Os tamanhos dos espectrogramas variam. Detalhes:")
        for caminho, tamanho in tamanhos.items():
            print(f"{caminho}: {tamanho}")

    if len(canais_unicos) == 1:
        print(f"Todas as imagens têm o mesmo número de canais: {canais_unicos.pop()}")
    else:
        print("As imagens têm diferentes números de canais. Detalhes:")
        for caminho, num_canais in canais.items():
            print(f"{caminho}: {num_canais} canais")

# Substitua pelo caminho da sua pasta de espectrogramas
caminho_pasta = "/content/espectrogramas"
verificar_tamanhos_canais_pasta(caminho_pasta)


Total de imagens verificadas: 250
Todos os espectrogramas têm o mesmo tamanho: (2325, 924)
Todas as imagens têm o mesmo número de canais: 4


In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, activation='relu', input_shape=(2325,924,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.GlobalAveragePooling2D())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 2323, 922, 32)       │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 1161, 461, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 1159, 459, 64)       │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 579, 229, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 577, 227, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 288, 113, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 11)                  │           1,419 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 111,179 (434.29 KB)

 Trainable params: 111,179 (434.29 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = cnn.fit(train_dataset, epochs=30, validation_data=val_dataset)

Epoch 1/30


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 11)